# EMTF++

## Table of Contents

- [Introduction](#Introduction)
  
  * [Version](#Version)
  * [Requirements](#Requirements)
  * [New forward muon detectors](#New-forward-muon-detectors)
  * [EMTF firmware, emulator, fastsim](#EMTF-firmware,-emulator,-fastsim)
  
- [Algorithm](#Algorithm)

  * [Primitive conversion](#Primitive-conversion)
  * [Pattern recognition](#Pattern-recognition)
  * [Track building](#Track-building)
  * [Parameter assignment](#Parameter-assignment)
  
- [Performance plots](#Performance-plots)

  * [Resolution](#Resolution)
  * [Efficiency](#Efficiency)
  * [Rates](#Rates)
  * [Robustness](#Robustness)

- [Future plans](#Future-plans)

  * Run 3 (2021-) configuration
  * Displaced muons
  * Extension to overlap region
  * Etc

## Introduction

### Version

- __v1.1.0__ (2018-11-06): Fixed the incorrect number of bunches used in the rate calculation: ~~1866~~ 2808

- __v1.0.0__ (2018-10-18): Reworked the patterns and the NN to reduce resource usage, without affecting the performance. Used as the starting point for (i) CMSSW integration. (ii) Firmware implementation.

### Requirements

- <span style="font-size: 120%">Maintain sensitivity to electroweak scale physics at higher luminosity and pileup of the HL-LHC</span>
  * Report all standalone muon coordinates and momenta in convention to facilitate __global correlation__ with tracks from the __Track Trigger__
    + The tracker will have far better p<sub>T</sub> resolution for rate reduction
  * Incorporate additional HL-LHC forward muon detectors to improve 
    + Efficiency, redundancy, and improved standalone p<sub>T</sub> measurement
  * Maintain standalone muon trigger (without track combination) for sufficiently high p<sub>T</sub> threshold
    + HL-LHC is “only” 3-4X higher lumi, and we increased the max L1 rate
- <span style="font-size: 120%">Add sensitivity to new physics scenarios, i.e. acceptance to __displaced muons and HSCPs__ from long-lived particle decays</span>
  * Additional patterns/logic and momentum assignment required (vertex constrained and not)

### New forward muon detectors

<img src="figures/cms_upg_o_g_b_ni_gem_re1_me0_grid_160229.png" width="800px"/>
<a href="figures/cms_upg_o_g_b_ni_gem_re1_me0_grid_160229.pdf">Download PDF</a>

- New forward detectors improve redundancy, efficiency and timing (only iRPC)
  * GEM detectors: GE1/1, GE2/1
  * iRPC detectors: RE3/1, RE4/1
  * ME0 detector
- New detectors also provide additional inputs and improved angular information (bend angles) for better correlation with TT tracks and better standalone muon p<sub>T</sub> measurement
- This requires increasing the bandwidth to the L1 muon electronics and adding more algorithm logic capability

### EMTF firmware, emulator, fastsim

While EMTF is really one algorithm, it is implemented in __two__ different ways: (i) firmware, written in Verilog and implemented in the MTF7 board; (ii) emulator, written in C++ and implemented as part of CMSSW software ([GitHub repo](https://github.com/cms-sw/cmssw/tree/master/L1Trigger/L1TMuonEndCap)). In the perfect world, they should be identical, but this is not always the case, and it is very important to recognize this fact when referring to the details of the EMTF algorithm.

In this document, I will try my best to be clear about which algorithm is being discussed. I'll denote the firmware algorithm as simply <span style="color: red; font-weight: bold">"firmware"</span>, and the emulator algorithm as <span style="color: red; font-weight: bold">"emulator"</span>. Whenever there is discrepancy, __firmware is always right__. Alex has written a description of the firmware algorithm [here](figures/EMU_TF_algorithm-1.pdf). 

In addition, in the course of developing the Phase-2 EMTF algorithm (namely EMTF++), I made another implementation in Python, and can be used outside of CMSSW. I'll denote this as <span style="color: red; font-weight: bold">"fastsim"</span>. The main reason I do this is because (i) it's very tedious to work with the emulator directly, and (ii) it allows me to use certain powerful Python libraries directly <sup>[[1]](#myfootnote1)</sup>. Basically, I have made simple ntuples and I can run Python scripts on them much more quickly (instead of going through the full-blown CMSSW). It allows me to make changes more quickly.

<a name="myfootnote1"><sup>1</sup></a>   I found the scientific stack in Python very powerful. These are the libraries I have used in my study: rootpy, NumPy, SciPy, Scikit-learn, Keras, Tensorflow.

At the time of writing, this is what we have:
- EMTF: firmware, emulator
- EMTF++: fastsim

My goal is to have the firmware and emulator versions of EMTF++ ready soon. (when?!)

## Algorithm

The EMTF system consists of 12 EMTF processors &mdash; 6 for each endcap, 1 for each 60&deg; trigger sector (at 15&deg;, 75&deg;, ...). Each sector works independently.

<img src="figures/trigger_sectors.png" width="500px"/>

To ensure coverage near the boundaries, each sector N also shares certain chambers with sector N+1. So, although each sector is nominally 60&deg;, it actually covers 70&deg; in the rings with 10&deg;-wide chambers, or 80&deg; in the rings with 20&deg;-wide chambers.
- ME1: chambers with CSC_ID=3,6,9, subsector=2 are shared
- ME2,3,4: chambers with CSC_ID=3,9 are shared

The EMTF algorithm starts by receiving the hits, or trigger primitives, and ends by producing muon tracks with certain information: (most importantly) p<sub>T</sub> , &eta;, &#981;, track quality, and its associated hits. I will break the EMTF algorithm into 4 building blocks. 

<img src="figures/algorithm_blocks.gv.svg" width="800px"/>

Each block may consist of smaller blocks. EMTF++ is an extension of the EMTF algorithm, and it follows the same design. In the following, I'll try to describe how each block works and what are the differences between EMTF vs EMTF++.

### Primitive conversion

In Phase-1, EMTF receives two types of trigger primitives: <span style="color: #31bd38">&#11035;</span> __CSC__ and <span style="color: #0096ff">&#11035;</span> __RPC__ hits. In Phase-2, EMTF++ will receive additional <span style="color: #e63600">&#11035;</span> __GEM__, <span style="color: #9437ff">&#11035;</span> __iRPC__ and <span style="color: #ffa25f">&#11035;</span> __ME0__ trigger primitives. 

The most important function here is to convert strip (half-strip, actually) and wire numbers to integer &#981; and &theta; units. The integer &#981; unit is a local coordinate defined within a sector. The scale of the unit is 1/60 = 0.0167&deg;, encoded as a 13-bit integer (0 to 8191). The position 0 corresponds to -22&deg; from the lower boundary of the sector (since Sep 2018). To convert from the integer &#981; unit to CMS global coordinate (in degrees):

\begin{align}
\phi_{\text{sector}} &= \phi_{\text{integer}} / 60 - 22 \\
\phi_{\text{CMS}} &= \phi_{\text{sector}} + 15 + (\text{sector}-1) \times 60 \\
& \quad \  \text{sector} \in \{1,2,3,4,5,6\}
\end{align}

The integer &theta; unit has a scale of 0.285&deg; and is encoded as a 8-bit integer (0 to 255). The position 0 corresponds to 8.5&deg;. To convert from the integer &theta; unit to CMS global coordinate (in degrees):

\begin{align}
\theta_{\text{CMS}} &= \theta_{\text{integer}} \cdot \left(\frac{45-8.5}{128}\right) + 8.5\\
\end{align}

The conversions from half-strip number to the integer &#981; unit and from wire number to the integer &theta; unit are done using simple linear equations. The constants (i.e. offsets & slopes) are derived offline and stored in the Block RAM in the firmware. There are additional corrections to the &theta; conversion for ME1/1 chambers with tilted wires, which I won't go into details here.

<span style="font-size: 120%; font-weight: bold">(i) CSC</span>

The CSC trigger primitive is called a Local Charge Track (LCT). It is a track segment reconstructed from up to 6 hits in the CSC detector. LCTs are built by the Trigger MotherBoard (TMB) and then sent to the Muon Port Card (MPC). The data are concentrated by MPC and then sent to EMTF.

CSC geometry is very interesting. When designing the trigger, one must pay attention to the following features:
- ME1 has 3 rings, and the chambers are 10&deg; wide. ME2,3,4 have 2 rings, and the ring 1 chambers are 20&deg; wide; the ring 2 chambers are 10&deg; wide.
- ME1 ring 1 is located at a different z position (~6m) compared to rings 2 & 3 (~7m). The magnetic field is very different for ring 1 because it is inside the solenoid ($B_z = 3.8{\text{T}}$). The wires in ring 1 are tilted by 29&deg; to compensate for the drift of the electrons in the strong magnetic field.
- ME1 ring 1 is further split into ME1/1a and ME1/1b at &eta;~2.05. They have different numbers of strips.
  * ME1/1a has 48 strips, ME1/1b has 64 strips, ME1/2 has 80 strips, ME1/3 has 64 strips, ME2,3,4 have 80 strips
- ME1 ring 3 is only 80% (?) efficient.
- Depending on the location of a chamber (even or odd chamber number, ME1,2 or ME3,4, positive or negative endcap), a chamber can be closer or further from the interaction point, and this can degrade the trigger performance if not taken into account.

See detector note [IN-2007/024](figures/IN2007_024.pdf) for more details.

The strips and wires are independent of each other. If there are two LCTs in a given chamber, we will get 2 strip numbers and 2 wire numbers. We do not know which strip number and which wire number to be paired up. Thus, we consider all the possibilities. Since there are at most 2 LCTs in a given chamber, there are at most 4 combinations to consider.

In <span style="display: inline-block; border-radius: .4rem; padding: 0.35rem .6rem; background: #3776ab; color: #ffffff; vertical-align: middle">EMTF++</span>, we will use the same primitive conversion for CSC that has been implemented in the firmware and in the emulator. But, we want to add a new bend variable, if possible.

### Pattern recognition

Zones ...

### Track building

### Parameter assignment

## Performance plots

### Resolution

### Efficiency

### Rates

### Robustness

## Future plans

In [1]:
#### Please ignore

In [2]:
if True:
  from IPython.core.display import HTML
  def css_styling():
      styles = '<style>.text_cell { font-family: "Helvetica Neue","Open Sans","Liberation Sans",Arial,sans-serif; }</style>'
      return HTML(styles)
  css_styling()

In [3]:
if False:
  import graphviz as gv
  d = gv.Digraph('algorithm_blocks', engine='dot')
  d.attr(rankdir='LR')

  d.attr('node', shape='plaintext', fontname='Helvetica, sans-serif', fontsize='14')
  d.node('I', 'hits')
  d.node('O', 'tracks')

  d.attr('node', shape='square', style='rounded,filled', color='#1C7C4C', fontcolor='white', fixed_size='true', width='1.3')
  d.node('A', 'Primitive\nConversion')
  d.node('B', 'Pattern\nRecognition')
  d.node('C', 'Track\nBuilding')
  d.node('D', 'Parameter\nAssignment')

  d.attr('edge', fontname='Helvetica, sans-serif', fontsize='12', arrowsize='0.6')
  #d.edges(['IA', 'AB', 'BC', 'CD', 'DO'])
  d.edge('I', 'A', label='')
  d.edge('A', 'B', label='conv.\nhits')
  d.edge('B', 'C', label='roads')
  d.edge('C', 'D', label='proto-\ntracks')
  d.edge('D', 'O', label='')

  d.format='svg'
  d.render()
  d